In [1]:
import pandas as pd

melb_nd = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/87856cc090b7a67a7a0689b0415c8331/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/melb_data_fe.zip')
melb_df = melb_nd.copy()
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [2]:
# Мы уже умеем строить таблицу, которая показывает зависимость медианной цены и площади здания от числа комнат:

melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


In [3]:
# Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type). 
# Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.

melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

Rooms  Type     
1      house        8.668655e+05
       townhouse    5.927045e+05
       unit         3.899289e+05
2      house        1.017238e+06
       townhouse    7.101585e+05
       unit         6.104905e+05
3      house        1.109233e+06
       townhouse    9.847087e+05
       unit         8.505963e+05
4      house        1.462283e+06
       townhouse    1.217092e+06
       unit         1.037476e+06
5      house        1.877327e+06
       townhouse    1.035000e+06
6      house        1.869508e+06
       unit         5.200000e+05
7      house        1.920700e+06
8      house        1.510286e+06
       unit         2.250000e+06
10     house        9.000000e+05
Name: Price, dtype: float64

In [4]:
# Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак 
# по строкам, а второй — по столбцам), а не в виде Series с иерархическими индексами, к результату чаще всего 
# применяют метод unstack(), который позволяет переопределить вложенный индекс в виде столбцов таблицы:

melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


In [5]:
# Построим ту же самую таблицу, но уже с использованием метода pivot_table.

melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',    fill_value=0
).round()

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


In [6]:
# А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. 
# Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), 
# а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день 
# был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


In [7]:
# Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от 
# типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, 
# нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую сводную таблицу, где 
# пропущенные значения заменим на 0:

melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
).round()

median                   mean                 
Type                        house townhouse unit   house townhouse   unit
Regionname                                                               
Eastern Metropolitan        674.0     234.0  203   717.0     269.0  330.0
Eastern Victoria            843.0       0.0  230  3109.0       0.0  295.0
Northern Metropolitan       460.0     134.0    0   619.0     317.0  495.0
Northern Victoria           724.0       0.0    0  3355.0       0.0    0.0
South-Eastern Metropolitan  630.0     240.0  199   664.0     212.0  358.0
Southern Metropolitan       586.0     246.0    0   570.0     279.0  466.0
Western Metropolitan        531.0     198.0   62   508.0     245.0  558.0
Western Victoria            600.0       0.0    0   656.0       0.0    0.0

In [8]:
#                            Многомерные таблицы
# Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) 
# и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и 
# столбцов будет стоять медианная цена объекта (Price):

melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)
    

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                   1244000            780000   
       townhouse                760000                 0   
       unit                     650000                 0   
S      house                   1127000            675000   
       townhouse                828000                 0   
       unit                     645750            492000   
SA     house                    932500            950000   
       townhouse                807500                 0   
       unit                          0                 0   
SP     house                   1050000            672500   
       townhouse                910000                 0   
       unit                     515000            400000   
VB     house                   1100000            712500   
       townhouse                892500                 0   
       unit                     500000                 0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                     900000             500000   
       townhouse                 632500                  0   
       unit                      410000                  0   
S      house                     920000             555000   
       townhouse                 750000                  0   
       unit                      525500                  0   
SA     house                     817500             540000   
       townhouse                 425000                  0   
       unit                      616000                  0   
SP     house                     900000             521000   
       townhouse                 690000                  0   
       unit                      470000                  0   
VB     house                    1050000             690000   
       townhouse                 640000                  0   
       unit                      450000                  0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                          865000                1725000   
       townhouse                     1190000                1055000   
       unit                           525000                 571250   
S      house                          883300                1611000   
       townhouse                      875000                1135000   
       unit                           606000                 655000   
SA     house                          880000                1390000   
       townhouse                           0                1141000   
       unit                                0                 580000   
SP     house                          770000                1521750   
       townhouse                      800000                1162500   
       unit                           601000                 550000   
VB     house                          850000                1800000   
       townhouse                           0                1250000   
       unit                           700000                 500000   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                    870000            630000  
       townhouse                670000                 0  
       unit                     360000                 0  
S      house                    870000            397500  
       townhouse                729000                 0  
       unit                     489000                 0  
SA     house                    772500                 0  
       townhouse                467500                 0  
       unit                     571000                 0  
SP     house                    865000            360000  
       t

In [9]:
# Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

# Выведем её столбцы с помощью атрибута columns:
pivot.columns
# В результате мы получаем объект MultiIndex

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [10]:
# Из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit, 
# просто последовательно обратившись по имени столбцов:

display(pivot['mean']['unit'])

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [11]:
# Аналогично производится и фильтрация данных. Например, если нам нужны регионы, в которых средняя площадь 
# здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:

mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

In [12]:
# Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index и обернуть результат в список:

print(list(filtered_pivot.index))

['Southern Metropolitan', 'Western Metropolitan']


In [13]:
# Примечание. На самом деле мультииндексные таблицы можно создавать и вручную. Давайте посмотрим на 
# синтаксис данной конструкции:

import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

white  up       0.822782
       down     0.532546
       right    0.659485
blue   up       0.465917
       down     0.972584
red    up       0.390606
       down     0.558071
       left     0.066475
dtype: float64

In [14]:
# Если посмотреть на индексы Series, можно увидеть, что они являются мультииндексами:

print(mser.index)

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )


In [15]:
# Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — для этого вложенный 
# список передаётся в параметр columns при инициализации таблицы:

mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

pen               paper          
                   1         2         1         2
white up   -0.500590 -0.144502  1.758280 -1.981506
      down -1.099634 -1.138147  0.799680  0.321998
red   up    1.077495  0.521506  1.388638 -3.015244
      down  0.644203  0.998448  0.027905 -0.739527

In [23]:
# Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от 
# типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации 
# признаков площадь здания наибольшая?

melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median'
)

Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126.0,126.0,141.0,177.0,126.0,216.5,126.0,126.0
townhouse,88.0,114.0,126.0,159.5,152.0,NaN,NaN,NaN,NaN
unit,69.5,110.0,126.0,126.0,NaN,171.0,NaN,126.0,NaN


In [32]:
# Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price) 
# от риелторского агентства (SellerG) и типа здания (Type).

pivot1 = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
    fill_value=0
)
pivot1_max = pivot1['unit'].max()
display(pivot1[pivot1['unit'] == pivot1_max].index[0])

'Nick'